# Merge metaclusters from pixie - Pixel level analysis

This very small Jupyter Notebook is supposed to be run, after the first step of pixel clustering has been run on the entire dataset. 

The user can manually merge metaclusters and rename them. After finishing this notebook, make sure to run XXX to apply the updated clusters to your pixel values and generate pixel maps.

In [4]:
# import required packages
import json
import os
from datetime import datetime as dt

import feather
import matplotlib.pyplot as plt
import pandas as pd
from alpineer import load_utils
from matplotlib import rc_file_defaults

from ark.phenotyping import (cell_cluster_utils, cell_meta_clustering,
                             cell_som_clustering, weighted_channel_comp)
from ark.utils import data_utils, example_dataset, plot_utils
from ark.utils.metacluster_remap_gui import (MetaClusterGui,
                                             colormap_helper,
                                             metaclusterdata_from_files)

# explicitly set cell_cluster_prefix to override datetime default
base_dir = "/gpfs/bwfor/work/ws/hd_gr294-mi_lunaphore_mcmicro/pixie_subset"
cell_cluster_prefix = "cell_clustering_0.05"
pixel_cluster_col = "pixel_meta_cluster_rename"

if cell_cluster_prefix is None:
    cell_cluster_prefix = dt.now().strftime('%Y-%m-%dT%H:%M:%S')

# define the base output cell folder
cell_output_dir = '%s_cell_output_dir' % cell_cluster_prefix
if not os.path.exists(os.path.join(base_dir, "pixie", cell_output_dir)):
    os.mkdir(os.path.join(base_dir, "pixie", cell_output_dir))
    
# define the paths to cell clustering files, explicitly set the variables to use custom names
cell_som_weights_name = os.path.join("pixie", cell_output_dir, 'cell_som_weights.feather')
cluster_counts_name = os.path.join("pixie", cell_output_dir, 'cluster_counts.feather')
cluster_counts_size_norm_name = os.path.join("pixie", cell_output_dir, 'cluster_counts_size_norm.feather')
weighted_cell_channel_name = os.path.join("pixie", cell_output_dir, 'weighted_cell_channel.feather')
cell_som_cluster_count_avg_name = os.path.join("pixie", cell_output_dir, 'cell_som_cluster_count_avg.csv')
cell_meta_cluster_count_avg_name = os.path.join("pixie", cell_output_dir, 'cell_meta_cluster_count_avg.csv')
cell_som_cluster_channel_avg_name = os.path.join("pixie", cell_output_dir, 'cell_som_cluster_channel_avg.csv')
cell_meta_cluster_channel_avg_name = os.path.join("pixie", cell_output_dir, 'cell_meta_cluster_channel_avg.csv')
cell_meta_cluster_remap_name = os.path.join("pixie", cell_output_dir, 'cell_meta_cluster_mapping.csv')

In [6]:
%matplotlib widget
rc_file_defaults()
plt.ion()

cell_mcd = metaclusterdata_from_files(
    os.path.join(base_dir, cell_som_cluster_count_avg_name),
    cluster_type='cell',
    prefix_trim=pixel_cluster_col + '_'
)
cell_mcd.output_mapping_filename = os.path.join(base_dir, cell_meta_cluster_remap_name)
cell_mcg = MetaClusterGui(cell_mcd, width=17)